In [ ]:

# leba3207

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier


import difflib
from functools import partial

Chargement des donnÃ©es des diffÃ©rents fichiers

In [ ]:

data_folder = 'data/'

journal_file = 'api_journal11-13-17.csv'
price_file = 'api_price11-13-17.csv'
influence_file = 'estimated-article-influence-scores-2015.csv'

journal = pd.read_csv(data_folder + journal_file, sep=',', encoding='latin1')
price = pd.read_csv(data_folder + price_file, sep=',', index_col=0)
influence = pd.read_csv(data_folder + influence_file, sep=',', index_col=0)


In [ ]:

def get_uniqueness_attributes(table):
    return table.nunique(axis=0)


def get_ratio_missing_values(table):
    return table.isnull().sum() * 100 / len(table)


def get_unique_values_of_attribute(table, header):
    return table[header].unique()


def lowercase_columns(table, headers):
    for header in headers:
        table[header] = table[header].str.lower()


# TODO: include headers specification
def get_df_duplicated_rows_dropped(table):
    return table.drop_duplicates()


def plot_categories_frequency(table, header):
    fig, ax = plt.subplots()
    table[header].value_counts()[0:5].plot(ax=ax, kind='bar')
    plt.title(f'FrÃ©quence d\'apparition des catÃ©gories de l\'attribut {header}')
    plt.show()


def get_mean_price_per_year():
    mean_price_per_year = {}
    for index, p in price.iterrows():
        year = p['date_stamp'].year
        if year in mean_price_per_year:
            mean_price_per_year[year] += p['price']
        else:
            mean_price_per_year[year] = p['price']

    for year, value in mean_price_per_year.items():
        mean_price_per_year[year] /= len(price[price['price'] != 0])
    return {k: v for k, v in sorted(mean_price_per_year.items(), key=lambda item: item[1], reverse=True)}


def rename_df_headers(table, dict_headers):
    return table.rename(columns=dict_headers)


def get_score_sequence_matching(s, c1, c2):
    return difflib.SequenceMatcher(None, s[c1], s[c2]).ratio()


# Question 1: Exploration-Description
## PrÃ©senter une description de chacun des attributs des 3 tables, avec des graphiques pour la visualisation des 
statistiques descriptives au besoin.

### Table journal

In [ ]:
# TODO: visualisations
# frÃ©quence des valeurs

In [ ]:

print(journal.head())

In [ ]:

"""
issn: identifiant du journal
Les valeurs de cet attribut semblent suivre un format particulier tel que: 4 digits - 4 digits

journal_name: nom textuel du journal
Les valeurs sont textuelles, ne suivant pas de valeurs catÃ©gorielles particuliÃ¨re Ã  priori.

pub_name: nom de l'Ã©diteur du journal
Les valeurs sont textuelles, ne suivant pas de valeurs catÃ©gorielles particuliÃ¨re Ã  priori.

is_hybrid: indique si le journal est hybride, ce qui signifie que c'est une revue sur abonnement dont certains articles
sont en libre accÃ¨s, comme l'indique le site http://flourishoa.org/about.

category: renseigne sur la/les catÃ©gorie(s) des sujets abordÃ©s par le journal 
Les valeurs sont textuelles et sont catÃ©gorielles. Chaque objet peut possÃ©der des valeurs multiples pour cet attribut. La sÃ©paration entre les diffÃ©rentes valeurs semblent
Ãªtre inconsistante.

url: indique l'adresse url du site du journal


Les attributs journal_name, pub_name et category Ã©tant des donnÃ©es textuelles trÃ¨s inconsistantes, je dÃ©cide avant tout
traitement et Ã©tude supplÃ©mentaire de transformer les valeurs en minuscule pour limiter au maximum l'inconsistence.
"""

In [ ]:

lowercase_columns(journal, ['journal_name', 'pub_name', 'category'])

In [ ]:

print(f"Valeurs uniques des attributs de journal:\n"
      f"{get_uniqueness_attributes(journal)}")
print(f"Ratio de valeurs vides pour les attributs de journal:\n"
      f"{get_ratio_missing_values(journal)}")

print(f"Valeurs possibles pour l'attribut is_hybrid de journal:\n"
      f"{get_unique_values_of_attribute(journal, 'is_hybrid')}")
print(f"Valeurs possibles pour l'attribut category de journal:\n"
      f"{get_unique_values_of_attribute(journal, 'category')}")

Les attributs category et url prÃ©sentent un nombre consÃ©quent de valeurs manquantes.
L'attribut issn prÃ©sente des valeurs uniques pour chacun de ses objets.

On remarque qu'il existe uniquement deux valeurs pour l'attribut is_hybrid (soit 1 soit 0). 

In [ ]:

print(f"Valeurs possibles de pub_name quand is_hybrid vaut 1:\n"
      f"{journal[journal['is_hybrid'] == 1]['pub_name'].unique()}")

In [ ]:

plot_categories_frequency(journal, 'pub_name')
plot_categories_frequency(journal, 'category')

### Table price

In [ ]:

print(price.head())

price: information du prix d'une publication pour le journal associÃ© Ã  une date prÃ©cise, en dollar US
Si celui-ci est Ã  0, on peut coompendre que celui-ci est gratuit

date_stamp: horodatage de l'information de prix d'une publication, en format annÃ©es-mois-jour

journal_id: identifiant du journal
Les valeurs semblent suivre consistantement un format du type: 4 digits - 4 digits

influence_id: identifiant de l'influence
Les valeurs suivent un format 4 digits.

url: indique l'adresse url du site de l'auteur

license: indique le type de license utilisÃ© par le journal pour les diffÃ©rents articles utilisÃ©s.


On convertit l'attribut date_stamp en type date.

In [ ]:

price['date_stamp'] = pd.to_datetime(price['date_stamp'], errors='coerce', format='%Y-%m-%d')

In [ ]:

print(f"Valeurs uniques des attributs de price:\n"
      f"{get_uniqueness_attributes(price)}")
print(f"Ratio de valeurs vides pour les attributs de price:\n"
      f"{get_ratio_missing_values(price)}")

print(f"Exemples de valeurs possibles pour l'attribut influence_id de price:\n"
      f"{get_unique_values_of_attribute(price, 'influence_id')[1:8]}")
print(f"Valeurs possibles pour l'attribut license de price:\n"
      f"{get_unique_values_of_attribute(price, 'license')}")

Les attributs influence_id, url et license prÃ©sentent une majoritÃ© de valeurs manquantes.

In [ ]:

mean_price_per_year = get_mean_price_per_year()

plt.bar(range(len(mean_price_per_year)), mean_price_per_year.values())
plt.xticks(range(len(mean_price_per_year)), mean_price_per_year.keys())
plt.title("Moyenne par annÃ©e des prix des publications")
plt.show()

### Table influence

In [ ]:

print(influence.head())

# TODO: proj_ai moyenne

journal_name: nom textuel du journal
Les valeurs sont textuelles, ne suivant pas de valeurs catÃ©gorielles particuliÃ¨re Ã  priori.

issn: identifiant du journal
Les valeurs de cet attribut semblent suivre un format particulier tel que: 4 digits - 4 digits

citation_count_sum: indique le nombre de citations du jounal

paper_count_sum: indique le nombre de citations des articles du jounal

avg_cites_per_paper: indique la moyenne des citations par papier qui sont contenus du journal

proj_ai: information sur le score d'influence des articles du journal

proj_ai_year: spÃ©cification de l'annÃ©e oÃ¹ l'information sur le score d'influence des articles du journal a Ã©tÃ© Ã©tablie

In [ ]:

print(f"Valeurs uniques des attributs de influence:\n"
      f"{get_uniqueness_attributes(influence)}")
print(f"Ratio de valeurs vides pour les attributs de influence:\n"
      f"{get_ratio_missing_values(influence)}")

print(f"Valeurs possibles pour l'attribut proj_ai_year de influence:\n"
      f"{get_unique_values_of_attribute(influence, 'proj_ai_year')}")

L'attribut proj_ai_year ne prÃ©sentant qu'une seule valeur nous indique que les valeurs de l'attribut proj_ai ont toutes
Ã©tÃ© Ã©tablies Ã  la mÃªme pÃ©riode. 
"""

# TODO: proj_ai sum in function of journal

# Question 2: PrÃ©traitement-ReprÃ©sentation
## A. Effectuer un prÃ©traitement des donnÃ©es pour supprimer les duplications et corriger les incohÃ©rences sâ€™il y en a.

### Table journal
Dans un premier temps, on Ã©limine les objets prÃ©sentant des objets dupliquÃ©s sur tous les attributs.
On se base sur l'attribut issn qui devrait Ãªtre unique pour chaque objet de la table, on vÃ©rifie son unicitÃ©.

In [ ]:

nb = len(journal)
journal = get_df_duplicated_rows_dropped(journal)
print(f"Nombre d'objets dupliquÃ©s Ã©liminÃ©s dans journal: {nb - len(journal)}")

check = np.logical_not(journal['issn'].duplicated().any())
print(f"UnicitÃ© de l'attribut issn dans la table journal: {check}")

### Table price
Etant donnÃ© que les index Ã©taient fournis dans le fichier original et qu'on les utilise afin d'indexer nos objets, 
on vÃ©rifie qu'il n'existe pas de duplicata.

Ensuite, on Ã©limine les objets prÃ©sentant des objets dupliquÃ©s sur tous les attributs.
Dans un second temps, dans la table price, les objets se doivent d'Ãªtre uniques selon deux attributs, date_stamp et
journal_id. S'ils ne le sont pas, alors ceux sont des objets dupliquÃ©s.

In [ ]:

check = np.logical_not(price.index.duplicated().any())
print(f"UnicitÃ© des indexes de la table price: {check}")

nb = len(price)
price = get_df_duplicated_rows_dropped(price)
print(f"Nombre d'objets dupliquÃ©s Ã©liminÃ©s dans price: {nb - len(price)}")

duplicated_rows = price[price[['date_stamp', 'journal_id']].duplicated(keep=False)]

Il existe des duplicata ambigus que l'on dÃ©cide de traiter un Ã  un.

Premier cas: un des objets prÃ©sente un prix nul, on dÃ©cide de choisir de l'Ã©liminer au profit de l'autre.

In [ ]:

print(duplicated_rows.iloc[0].fillna(0) == duplicated_rows.iloc[1].fillna(0))
price = price.drop(duplicated_rows.index.values[0])

DeuxiÃ¨me cas: leur valeur du prix est diffÃ©rente d'un lÃ©ger Ã©cart, on dÃ©cide de garder la deuxiÃ¨me de maniÃ¨re
arbitraire

In [ ]:

print(duplicated_rows.iloc[40].fillna(0) == duplicated_rows.iloc[41].fillna(0))
price = price.drop(duplicated_rows.index.values[40])

TroisiÃ¨me cas: seule la valeur de license est diffÃ©rente, on dÃ©cide de garder la premiÃ¨re de maniÃ¨re arbitraire.

In [ ]:

for i in range(3, 39, 2):
    price = price.drop(duplicated_rows.index.values[i])

### Table influence
Etant donnÃ© que les index Ã©taient fournis dans le fichier original et qu'on les utilise afin d'indexer nos objets, 
on vÃ©rifie qu'il n'existe pas de duplicata.

On se base sur l'attribut issn qui devrait Ãªtre unique pour chaque objet de la table, on vÃ©rifie son unicitÃ©.

In [ ]:

nb = len(influence)
influence = get_df_duplicated_rows_dropped(influence)
print(f"Nombre d'objets dupliquÃ©s Ã©liminÃ©s dans influence: {nb - len(influence)}")

check = np.logical_not(influence['issn'].duplicated().any())
print(f"UnicitÃ© de l'attribut issn dans la table influence: {check}")

Les attributs citation_count_sum, paper_count_sum, avg_cites_per_paper et proj_ai prÃ©sentent des valeurs nulles que l'on
dÃ©cide d'attribuer Ã  0.

In [ ]:

# headers = ['citation_count_sum', 'paper_count_sum', 'avg_cites_per_paper', 'proj_ai']
# for header in headers:
#     influence[header] = influence[header].fillna(0)

## Merge
Afin de simplifier les opÃ©rations, on gÃ©nÃ¨re une seule table reprenant les informations des trois tables.
On vÃ©rifie d'abord si les identifiants communs aux diffÃ©rentes tables sont prÃ©sentes dans les tables Ã  merger.
En premier, on vÃ©rifie si les valeurs de l'attribut issn de influence sont existantes dans l'attribut du mÃªme nom 
dans journal.
De mÃªme, on vÃ©rifie les valeurs de l'attribut journal_id de price sont existantes dans l'attribut issn dans journal. 

In [ ]:

check = influence['issn'].isin(journal['issn']).any()
print(f"Pas de valeur d'issn manquante dans journal par rapport Ã  influence : {check}")

check = price['journal_id'].isin(journal['issn']).any()
print(f"Pas de valeur d'issn manquante dans journal par rapport Ã  price : {check}")

On applique maintenant le merge des trois tables en deux Ã©tapes. D'abord, on merge influence dans journal, puis price
est ensuite mergÃ© dans le rÃ©sultat du premier merge.

In [ ]:

price = rename_df_headers(price, {"journal_id": "issn", "url": "url_autor"})
journal = rename_df_headers(journal, {"url": "url_journal"})

temp = pd.merge(journal, influence, on='issn', how='outer')
check = len(temp[temp['journal_name_x'] != temp['journal_name_y']])
print(f"Nombre d'aberrances entre les valeurs journal_name des tables journal et influence: {check}")
temp = temp.drop(columns=['journal_name_y'])
temp = rename_df_headers(temp, {"journal_name_x": "journal_name"})

print(f"Valeurs uniques des attributs de temp:\n"
      f"{get_uniqueness_attributes(temp)}")

data = pd.merge(temp, price, on=['issn'], how='outer')
data = get_df_duplicated_rows_dropped(data)

print(f"Valeurs uniques des attributs de data:\n"
      f"{get_uniqueness_attributes(data)}")
print(f"Ratio de valeurs vides pour les attributs de data:\n"
      f"{get_ratio_missing_values(data)}")

On s'assure bien que les valeurs de l'attribut issn de la nouvelle date (data) sont uniques.

## B. Y-a-t-il une corrÃ©lation entre les catÃ©gories de journaux (attribut category) et les coÃ»ts de publication (attribut price)? Justifier la rÃ©ponse.

In [ ]:

labelled_data = data[data['category'].notna()]
data_to_predict = data[data['category'].isna()]

In [ ]:

labelled_data['category'] = labelled_data['category'].str.replace(r'[\.\|&] | [\.\|&] ', '.', regex=True)
category_dummies = labelled_data['category'].str.get_dummies(sep='.')
category_dummies = category_dummies.add_prefix('category_')

In [ ]:

labelled_data = pd.concat([labelled_data, category_dummies], axis=1) \
    # .drop(columns=['category'])

In [ ]:

categories_correlation = {}

for header in category_dummies.columns:
    corr = labelled_data[header].corr(labelled_data['price'])
    if abs(corr) > 0.1:
        categories_correlation[header] = corr

fig, ax = plt.subplots()
plt.bar(categories_correlation.keys(), categories_correlation.values())
plt.setp(ax.get_xticklabels(), rotation=30, horizontalalignment='right')
plt.title(f'CatÃ©gories prÃ©sentant des corrÃ©lations fortes avec l\'attribut prix et leurs valeurs')
plt.show()

## C. Construire un modÃ¨le pour prÃ©dire les valeurs de catÃ©gorie de journaux manquantes de la faÃ§on la plus prÃ©cise 
possible (cela inclut la sÃ©lection dâ€™attributs informatifs, le choix et le paramÃ©trage dâ€™un modÃ¨le de classification, 
le calcul du score du modÃ¨le, lâ€™application du modÃ¨le pour prÃ©dire les catÃ©gories manquantes). Justifier les choix 
effectuÃ©s.

In [ ]:

labelled_data = labelled_data[labelled_data['price'].notna()]

headers = ['citation_count_sum', 'paper_count_sum', 'avg_cites_per_paper', 'proj_ai', 'price']
labelled_data = labelled_data.dropna(axis=0, subset=headers)

In [ ]:

journal_name_score_match_category = pd.Series()
publisher_name_score_match_category = pd.Series()

labelled_data['journal_name_sm_category'] = labelled_data.apply(partial(get_score_sequence_matching, c1='journal_name',
                                                                        c2='category'), axis=1)
labelled_data['pub_name_sm_category'] = labelled_data.apply(partial(get_score_sequence_matching, c1='pub_name',
                                                                    c2='category'), axis=1)
# %%

labelled_data = labelled_data.drop(columns=['category'])
print(f'size labelled_data before splitting: {labelled_data.shape[0]}')

In [ ]:
attributes_of_interest = ['citation_count_sum', 'paper_count_sum', 'avg_cites_per_paper', 'proj_ai', 'price',
                          # 'date_stamp',
                          'journal_name_sm_category', 'pub_name_sm_category']
headers = category_dummies.columns.tolist()

In [ ]:

clfs = {'RandomForestClassifier': RandomForestClassifier()}

best_model = {'name': '', 'score': 0, 'model': None}
for name, clf in clfs.items():
    # for i in range(13, 16): TODO
    for i in range(14, 15):
        X_train, X_test, y_train, y_test = train_test_split(labelled_data.drop(columns=headers),
                                                            labelled_data[category_dummies.columns],
                                                            test_size=0.33, random_state=42)
        X_train = X_train[attributes_of_interest]
        X_test = X_test[attributes_of_interest]
        y_train = y_train.to_numpy()

        clf.set_params(max_depth=i)

        print(f'ModÃ¨le {name} {i}')
        print('## Entrainement ##')
        classifier = MultiOutputClassifier(clf, n_jobs=-1)
        classifier.fit(X_train, y_train)
        train_score = classifier.score(X_train, y_train)
        print(f'Score d\'entraÃ®nement: {train_score}')

        print('## Test ##')
        test_predictions = classifier.predict(X_test)
        test_score = classifier.score(X_test, y_test)
        print(f'Score de test: {test_score}')

        if test_score > best_model.get('score'):
            best_model['name'], best_model['score'], best_model['model'] = name + ' ' + str(i), test_score, clf

print(f"Le modÃ¨le prÃ©sentant le meilleur score est {best_model.get('name')} avec {best_model.get('score')}")


RÃ©sultats des diffÃ©rents essais:

ModÃ¨le KNeighborsClassifier 3
## Entrainement ##
Score d'entraÃ®nement: 0.6727145847871598
## Test ##
Score de test: 0.3988684582743989

ModÃ¨le RandomForestClassifier 14
## Entrainement ##
Score d'entraÃ®nement: 0.9993021632937893
## Test ##
Score de test: 0.7199434229137199

# Question 3: RÃ©grÃ©ssion-clustering
## A. Supprimer tous les attributs ayant plus de 50% de donnÃ©es manquantes.